<a href="https://colab.research.google.com/github/OdysseusPolymetis/colabs_for_nlp/blob/main/2_analyse_textuelle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**ANALYSE TEXTUELLE, LES BASES**

In [ ]:
!pip install stanza

##Qu'est-ce que c'est que l'analyse textuelle (automatique), rapidement
En fait ça peut toucher beaucoup de domaines. Ici, on va en voir plusieurs, entre autres la **tokenisation**, la **lemmatisation**, le **postagging**. J'appellerai ça les étapes de "pre-processing", de travail préliminaire.
<br>En effet, on a très souvent besoin, pour faire des choses plus poussées, de ces étapes pour éviter de créer des biais dans les analyses qui vont suivre.
<br>Voici quelques exemples de biais possibles, si l'on ne passe pas par certaines de ces étapes de pré-traitement.

Prenons l'exemple des nuages de mots. C'est globalement facile à faire, et ça permet souvent (de manière parfois un peu artificielle) d'illustrer son propos.
Ici voyons ce qu'on obtient si on fait un nuage de mots sur les _Trois Mousquetaires_.

![](https://drive.google.com/uc?export=view&id=1g8RLOXodDDGBgfR6tJ-jvVG1JHTdz65f)

Ce nuage est fait avec des données brutes. C'est-à-dire qu'on a juste injecté les mots du texte, tels quels. Globalement, on ne peut pas en faire grand chose et ce n'est pas représentatif. Pourquoi ? Déjà parce que chaque mot est encore fléchi, et donc pour chaque "fut", "est", "sera", chaque mot est compté comme un mot à part, et non comme une forme de "être".
<br>Une première solution est donc de **lemmatiser**.

![](https://drive.google.com/uc?export=view&id=10vv3sdsqLMBMXYW5-YAKp9ztgYq_sJDs)

C'est déjà mieux, mais ça n'est pas fou non plus. Il y a encore de nombreux mots dont on se fiche complètement. Exemple, "faire", "plus", etc.
<br>On a donc encore une étape, celle d'enlever les **mots-outils**, ou "stopwords", et la ponctuation. Les mots-outils sont les mots qui n'ont qu'un faible poids sémantique (qui peuvent avoir leur importance pour une étude dédiée, certes), mais qui font du bruit lors d'une analyse statistique de base.
<br>Enlevons-les.

![](https://drive.google.com/uc?export=view&id=1c9PxvrkXDTNZTC7QOGNCi2qOQ5_fI9th)

##Comment on pré-traite ?
Il y a plusieurs écoles, et donc aussi plusieurs modules pour le faire.
<br>D'expérience personnelle, il y a les modules rapides et qui donnent des résultats plus mitigés, et des modules plus longs qui sont généralement meilleurs (mais pas forcément).
<br>Ici, je vais vous montrer trois modules avec des qualités et des défauts, pour le français (mais ils ont aussi des modèles en d'autres langues).

##<center>**LE TAL : TOKENISATION, LEMMATISATION, POSTAGGING**</center>

Nous allons tester un outil qui permet de faire ces trois opérations, **`stanza`**. Il en existe beaucoup d'autres.

In [ ]:
preface="Tant qu’il existera, par le fait des lois et des mœurs, une damnation sociale créant artificiellement, en pleine civilisation, des enfers, et compliquant d’une fatalité humaine la destinée qui est divine ; tant que les trois problèmes du siècle, la dégradation de l’homme par le prolétariat, la déchéance de la femme par la faim, l’atrophie de l’enfant par la nuit, ne seront pas résolus ; tant que, dans de certaines régions, l’asphyxie sociale sera possible ; en d’autres termes, et à un point de vue plus étendu encore, tant qu’il y aura sur la terre ignorance et misère, des livres de la nature de celui-ci pourront ne pas être inutiles."

In [ ]:
!gdown 1GEgd5cQoJkTm5PRWfixxOKHe3uOlxFqo

Downloading...
From: https://drive.google.com/uc?id=1GEgd5cQoJkTm5PRWfixxOKHe3uOlxFqo
To: /content/miserables.txt
100% 3.17M/3.17M [00:00<00:00, 11.8MB/s]


In [ ]:
filepath_of_text = "/content/miserables.txt"

In [ ]:
full_text = open(filepath_of_text, encoding="utf-8").read()

##**stanza (ancien Stanford CoreNLP)**

Personnellement, c'est l'analyseur que j'utilise quand je n'ai pas besoin de faire de représentations graphiques de mes résultats. Il est rapide et efficace.

Je mentionne `stanza` en particulier pour trois raisons :
<br>- d'abord parce qu'il dispose d'un très grand nombre de modèles de langue, et pas forcément des langues très répandues,
<br>- ensuite parce qu'il est très rapide, et niveau performance tout à fait satisfaisant pour les gros corpus,
<br>- enfin parce que je le trouve facile à manipuler et à implémenter.

Mais il faut garder en tête qu'il en existe bien d'autres qui fonctionnent vraiment très bien, avec un nombre de modèles qui se multiplie. Je pense au tagger de BERT,  ou de `flair` entre autres. Mais ça nécessite d'être un peu plus aguerri.

Là encore, il existe plusieurs modèles rien que pour le français (je vous mets ici la [liste des modèles](https://stanfordnlp.github.io/stanza/performance.html) dans d'autres langues), mais le modèle par défaut peut être appelé avec `fr`.
<br>C'est un modèle sur du français moderne, donc ne fonctionne pas avec l'ancien français etc. (mais un modèle existe aussi pour l'ancien français, appelé par défaut avec `fro`).

In [ ]:
import stanza
stanza.download('fr')

On commence par lui spécifier une Pipeline, c'est-à-dire qu'on lui signifie quels processeurs il va devoir mobiliser pour les opérations suivantes et en quelle langue. L'avantage de cette opération est qu'on ne mobilise pas l'artillerie lourde quand on veut faire des opérations simples).

In [ ]:
nlp_stanza = stanza.Pipeline(lang='fr', processors='tokenize,mwt,pos,lemma')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: fr (French):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


Maintenant, nous pouvons lancer le TAL sur la préface.

In [ ]:
miserables_stanza=nlp_stanza(preface)

Voyons maintenant ses résultats.

In [ ]:
for sent in miserables_stanza.sentences:
  for token in sent.words:
    print(token.text + ' - ' + token.lemma + ' - ' + token.pos)

Globalement ça marche d'enfer.

##<center>**LES ENTITES NOMMEES**</center>

Les **entités nommées** sont des éléments textuels connus et catégorisables. Par exemple, un NER (Named Entity Recognition) qui fonctionnerait bien comprendrait "la ville aux sept collines" comme un nom de lieu, et comme Rome.

##**stanza**

**`stanza`** aussi peut permettre de façon beaucoup plus rapide d'obtenir les NER. Ne serait-ce que parce que, par défaut, il demande de définir une Pipeline (un processus qui permet de ne pas faire toutes les opérations en même temps).

In [ ]:
stanza_ner = stanza.Pipeline(lang='fr', processors='tokenize,ner')

In [ ]:
doc = stanza_ner("qui avaient couru sur son compte au moment où il était arrivé dans le diocèse. Vrai ou faux, ce qu'on dit des hommes tient souvent autant de place dans leur vie et surtout dans leur destinée que ce qu'ils font. M. Myriel était fils d'un conseiller au parlement d'Aix ; noblesse de robe.")

In [ ]:
print(*[f'entity: {ent.text}\ttype: {ent.type}' for ent in doc.ents], sep='\n')

entity: M. Myriel	type: PER
entity: Aix	type: LOC


Les résultats sont bons pour ce passage, semble-t-il.